<a href="https://colab.research.google.com/github/cchummer/ml-dl-scratch/blob/main/classy_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np, pandas as pd

class myRandomForestClassifier:
  '''
  My custom random forest classifier.

  Properties:
  forest: Contains the list of trees making up the forest
  feature_importances: Contains the dictionary of calculated feature importances of the forest

  Main functions:
  fit(): Create a forest from the given training data
  predict(): Make a prediction on given samples
  '''

  def __init__(self, min_samples_leaf=50, n_trees=100, max_tree_depth=10, tree_sample_size=.75, branch_sample_size=.75):
    '''
    Initialize default values if none passed

    tree_sample_size: percent of training dataset each tree should randomly choose to train on (value of 1 means every tree gets whole set)
    branch_sample_size: percent of features/columns of training dataset each split/branch within each tree should randomly choose to split between (value of 1 means every split can see every feature)
    '''

    self.min_samples_leaf = min_samples_leaf
    self.n_trees = n_trees
    self.max_tree_depth = max_tree_depth
    self.tree_sample_size = tree_sample_size
    self.branch_sample_size = branch_sample_size

    # Will hold our forest and feature importances once created
    self._forest = []
    self._feature_importances = {}

  @property
  def forest(self):
    return self._forest

  @property
  def feature_importances(self):
    return dict(sorted(self._feature_importances.items(), key = lambda pair: pair[1], reverse = True))

  def node_gini(self, ys):
    '''
    Calculate a node's gini impurity. Also return the node size to weigh node's gini against sibling

    ys: Array of node's dependent variables/class labels

    Returns: Node gini, and number of samples in node
    '''

    if len(ys) == 0:
      return 0, 0  # Consider alternatives

    unique_classes, class_counts = np.unique(ys, return_counts=True)
    probabilities = class_counts / len(ys)
    gini = 1 - np.sum(probabilities**2)
    return gini, len(ys)

  def split_gini(self, ys1, ys2):
    '''
    Calculates the gini score of a potential split, a weighted average of the children node's gini scores

    ys1: Child node 1's class labels
    ys2: Child node 2's class labels

    Returns: Split gini value
    '''

    gini1, len1 = self.node_gini(ys1)
    gini2, len2 = self.node_gini(ys2)

    weighted_gini = (gini1 * len1 + gini2 * len2) / (len1 + len2)
    return weighted_gini

  def find_column_split_point(self, parent_xs, col, parent_ys):
    '''
    Helper function to find_df_split_point, finds the optimal split point for a specific column

    Returns: Calculated optimal column, cutoff point, and resulting split gini value
    '''

    unq = parent_xs[col].dropna().unique()
    unq_scores = []

    # Loop through unique values of column
    for c in unq:
      mask = parent_xs[col] <= c
      ys1 = parent_ys[mask]
      ys2 = parent_ys[~mask]

      unq_scores.append(self.split_gini(ys1, ys2))

    idx = unq_scores.index(min(unq_scores))

    return col, unq[idx], unq_scores[idx]

  def find_df_split_point(self, parent_xs, parent_ys):
    '''
    Find the optimal split point in the given sample set by split gini value

    Returns: Calculated optimal column, cutoff point, and resulting split gini value
    '''

    # Loop through each column, finding the best split point for each
    split_candidates = [self.find_column_split_point(parent_xs, col, parent_ys) for col in parent_xs.columns]

    # Now decide best column
    return min(split_candidates, key = lambda x: x[2]) # Score itself is in the third element returned by find_column_split_point

  def perform_binary_split(self, parent_xs, parent_ys, col, cutoff):
    '''
    Performs a binary split as determined by the given column and cutoff

    col: Column to split on
    cutoff: Value (in the given column) which samples having a value less than or equal to will be placed in child1, and samples having a value greater than the cutoff will be placed in child2

    Returns: 4 child dataframes: child1x, child1y, child2x, child2y
    '''

    child1x = parent_xs.loc[parent_xs[col] <= cutoff]
    child1y = parent_ys.loc[parent_xs[col] <= cutoff]

    child2x = parent_xs.loc[parent_xs[col] > cutoff]
    child2y = parent_ys.loc[parent_xs[col] > cutoff]

    return child1x, child1y, child2x, child2y

  def build_tree(self, X, y, max_depth, min_samples_leaf, branch_sample_size, depth=0):
    '''
    Builds one individual decision tree for use in the forest

    Returns: Tree in dictionary format
    '''

    # Calculate + store for feature importance calculation later
    cur_node_gini, node_size = self.node_gini(y)

    # If node is pure or max depth is reached or fewer samples than min_samples_leaf
    if len(np.unique(y)) == 1 or depth >= max_depth or len(y) < min_samples_leaf:
      return {
          'class': np.argmax(np.bincount(y)), # Most prominent dependent in a final node is considered the class predicted
          'node_gini' : cur_node_gini,
          'node_size' : node_size,
          'leaf': True
          }

    else:

      # Randomly sample features for consideration based on branch_sample_size. Value of 1 means all features considered for every split, .75 would be 75%, etc
      num_total_features = X.shape[1]
      num_subset = int(num_total_features * branch_sample_size)

      feature_idxs = list(range(num_total_features))
      subset_idxs = np.random.choice(feature_idxs, num_subset, replace = False)

      X_subset = X.iloc[:, subset_idxs]

      split_column, cutoff, split_gini = self.find_df_split_point(X_subset, y)
      X_left, y_left, X_right, y_right = self.perform_binary_split(X, y, split_column, cutoff)

      return {
          'split_column': split_column,
          'cutoff': cutoff,
          'node_size' : node_size,
          'node_gini' : cur_node_gini,
          'split_gini' : split_gini,
          'left': self.build_tree(X_left, y_left, max_depth, min_samples_leaf, branch_sample_size, depth + 1), # Increment depth and call again on child
          'right': self.build_tree(X_right, y_right, max_depth, min_samples_leaf, branch_sample_size, depth + 1),
          'leaf': False # Not yet a leaf, still a branch lol
          }

  def calc_feature_importances(self, forest):
    '''
    Calculates features importances by gini impurity decrease

    Returns: Dicionary holding normalized feature importance scores
    '''

    # Will hold aggregated impurity decreases attributed to each feature we come across. NOTE: Currently will not report on features not present in node via any random sampling of features per split
    impurity_decreases = {}

    for tree in forest:

      # Used to recursively traverse a tree's nodes
      def traverse_node_impurity(node):

        # Leaf node reached
        if 'left' not in node or 'right' not in node:
          return

        cur_feature = node['split_column']

        impurity_reduction = (node['node_gini'] - node['split_gini']) * node['node_size']

        # Add the calculated impurity reduction to the corresponding dictionary value
        impurity_decreases[cur_feature] = impurity_decreases.get(cur_feature, 0) + impurity_reduction

        # Traverse the children
        traverse_node_impurity(node['left'])
        traverse_node_impurity(node['right'])

      # Traverse each tree in the forest
      traverse_node_impurity(tree)

    # Now normalize decreases
    total_impurity_decrease = sum(impurity_decreases.values())

    for feature_name in impurity_decreases.keys():
      impurity_decreases[feature_name] /= total_impurity_decrease

    # Sort in descending order of importance, just for presentation
    return impurity_decreases

  def fit(self, X, y):
    '''
    Takes X and y pandas DataFrame objects, builds forest around them and calculates feature importances
    '''

    n_samples = len(y)
    n_sample_subset = int(n_samples * self.tree_sample_size)

    idxs = [np.random.choice(n_samples, n_sample_subset, replace = False) for i in range(self.n_trees)]

    self._forest = [self.build_tree(X.iloc[idx], y.iloc[idx], self.max_tree_depth, self.min_samples_leaf, self.branch_sample_size) for idx in idxs]
    self._feature_importances = self.calc_feature_importances(self.forest)

  def tree_predict_sample(self, tree, sample):
    '''
    Helper function to tree_predict. Handles one sample at a time, recursively passing it through the given tree

    Returns: A single class prediction
    '''

    if tree['leaf']: # == True
      return tree['class']

    else:
      if sample[tree['split_column']] <= tree['cutoff']:
        return self.tree_predict_sample(tree['left'], sample)
      else:
        return self.tree_predict_sample(tree['right'], sample)

  def tree_predict(self, tree, X):
    '''
    Helper function to the main predict function. Passes a given group of samples' through the single given tree.

    Returns: List of class predictions, one per sample
    '''

    return [self.tree_predict_sample(tree, sample) for _, sample in X.iterrows()]

  def predict(self, X):
    '''
    Main prediction function. Takes a majority vote of the trees in self.forest for each sample

    Returns: List of predicted classes, one per sample given
    '''

    # This will return a list of lists. One list per tree in the forest, which contains one prediction per row in X
    indiv_preds = [self.tree_predict(i, X) for i in self.forest]

    # Stack the list of lists of predictions into a 2D array. Each column now represents a single tree's predictions, and rows give an index to the sample (AKA row of X) to which the predictions correspond
    stacked_preds = np.stack(indiv_preds, axis=1)

    # For each sample, we will tally the class predictions across the trees and choose the class with most votes/occurances
    agg_preds = []

    for i in range(stacked_preds.shape[0]):

      # Grab current row
      cur_sample_preds = stacked_preds[i, :]
      class_counts = {}

      for cur_pred in cur_sample_preds:
        class_counts[cur_pred] = class_counts.get(cur_pred, 0) + 1

      # Store our final nominee for the row/sample
      agg_preds.append(max(class_counts, key=class_counts.get))

    return agg_preds

  def __str__(self):
    '''
    Print out feature importances by default
    '''

    return f'{self.feature_importances}'

# New Section

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML+DL/kaggle_titanic_train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# Same basic preprocessing

modes = df.mode().iloc[0]

def proc_data(df):
    df['Fare'] = df.Fare.fillna(0)
    df.fillna(modes, inplace=True)
    df['LogFare'] = np.log1p(df['Fare'])
    df['Embarked'] = pd.Categorical(df.Embarked)
    df['Sex'] = pd.Categorical(df.Sex)

proc_data(df)

cats=["Sex","Embarked"]
conts=['Age', 'SibSp', 'Parch', 'LogFare',"Pclass"]


dep="Survived"

In [ ]:
from sklearn.model_selection import train_test_split

#np.random.seed(42)
trn_df,val_df = train_test_split(df, test_size=0.25)
trn_df[cats] = trn_df[cats].apply(lambda x: x.cat.codes)
val_df[cats] = val_df[cats].apply(lambda x: x.cat.codes)

In [ ]:
# Split independents and dependent dataframes
def xs_y(df):
    xs = df[cats+conts].copy()
    return xs,df[dep] if dep in df else None

trn_xs,trn_y = xs_y(trn_df)
val_xs,val_y = xs_y(val_df)

# New Section

In [ ]:
custo_forest = myRandomForestClassifier()

In [ ]:
custo_forest.fit(trn_xs, trn_y)

In [ ]:
custo_forest.feature_importances

{'Sex': 0.560374437262397,
 'LogFare': 0.1464205559164615,
 'Pclass': 0.13681088769516753,
 'Age': 0.10110839825890124,
 'Embarked': 0.030833739031037925,
 'SibSp': 0.019214224812956072,
 'Parch': 0.005237757023078758}

In [ ]:
custo_forest.forest

In [ ]:
test_val_preds = custo_forest.predict(val_xs)

In [ ]:
test_train_preds = custo_forest.predict(trn_xs)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(trn_y, test_train_preds)

0.1497005988023952

In [ ]:
mean_absolute_error(val_y, test_val_preds)

0.17040358744394618

In [ ]:
test_val_preds